In [1]:
import requests
import json
import pandas as pd
from pprint import pprint
from citipy import citipy
import matplotlib.pyplot as plt
import numpy as np
import gmaps
import os
import csv
api_key = os.environ.get('weather_key')
gmap_key = os.environ.get('gmap_key') 

In [2]:

weather_dt = pd.read_csv('weather_dt.csv')
weather_dt.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Garoowe,97,SO,1628090939,34,8.4054,48.4845,88.81,22.44
1,1,Sīwah,0,EG,1628090939,13,29.2041,25.5195,106.56,6.98
2,2,Konëvo,7,RU,1628090940,41,62.1215,39.3192,66.20,11.18
3,3,Poum,99,NC,1628090940,71,-20.2333,164.0167,70.86,7.83
4,4,Ushuaia,75,AR,1628090703,41,-54.8000,-68.3000,55.06,18.41


In [3]:
gmaps.configure(api_key=gmap_key)

locations = weather_dt[["Lat", "Lng"]]

humidity = weather_dt["Humidity"]

In [4]:
heat_map=gmaps.figure(center=(46.0, -5.0), zoom_level=2)

heat_layers = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

heat_map.add_layer(heat_layers)
heat_map

Figure(layout=FigureLayout(height='420px'))

In [5]:
narrowed_dt = weather_dt.loc[(weather_dt["Wind Speed"] <= 10) & (weather_dt["Cloudiness"] == 0) & \
                                   (weather_dt["Max Temp"] >= 70) & (weather_dt["Max Temp"] <= 80)].dropna()

narrowed_dt

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
375,375,Hasaki,0,JP,1628091108,90,35.7333,140.8333,75.49,1.97
434,434,San Fernando del Valle de Catamarca,0,AR,1628090905,47,-28.4696,-65.7852,74.59,4.00
452,452,Waverley,0,ZA,1628091135,28,-25.7023,28.2591,71.31,1.14
509,509,San Lorenzo,0,PY,1628091154,40,-25.3333,-57.5333,72.54,4.61


In [6]:
hotel_dt = narrowed_dt.loc[:,["City","Country","Lat","Lng"]]

hotel_dt["Hotel Name"] = ""
hotel_dt

,City,Country,Lat,Lng,Hotel Name
375,Hasaki,JP,35.7333,140.8333,
434,San Fernando del Valle de Catamarca,AR,-28.4696,-65.7852,
452,Waverley,ZA,-25.7023,28.2591,
509,San Lorenzo,PY,-25.3333,-57.5333,


In [7]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : gmap_key}

In [9]:
for index, row in hotel_dt.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(url, params=params).json()
    
    results = response['results']
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_dt.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing results... skipping.")
        
    print("---")

print("---End Search---")

Retrieving Results for Index 375: Hasaki.
Closest hotel in Hasaki is Villa Umi To Mori.
---
Retrieving Results for Index 434: San Fernando del Valle de Catamarca.
Closest hotel in San Fernando del Valle de Catamarca is Amerian Catamarca Park Hotel.
---
Retrieving Results for Index 452: Waverley.
Closest hotel in Waverley is Sheraton Pretoria Hotel.
---
Retrieving Results for Index 509: San Lorenzo.
Closest hotel in San Lorenzo is Factoria Hotel.
---
---End Search---


In [10]:
hotel_dt

,City,Country,Lat,Lng,Hotel Name
375,Hasaki,JP,35.7333,140.8333,Villa Umi To Mori
434,San Fernando del Valle de Catamarca,AR,-28.4696,-65.7852,Amerian Catamarca Park Hotel
452,Waverley,ZA,-25.7023,28.2591,Sheraton Pretoria Hotel
509,San Lorenzo,PY,-25.3333,-57.5333,Factoria Hotel


In [11]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotels= [info_box_template.format(**row) for index, row in hotel_dt.iterrows()]
locations = hotel_dt[["Lat", "Lng"]]

In [12]:
markers = gmaps.marker_layer(locations, info_box_content = hotels)
heat_map.add_layer(markers)
heat_map

Figure(layout=FigureLayout(height='420px'))